In [ ]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [ ]:
df = pd.read_csv("df_ready.csv")

df_filter = df.filter(
    items = [
        "AgeMonthsInsemination",
        "Milk",
        "Fat",
        "Proteins",
        "ThiMax_dayless_5",
        "ThiMax_dayless_30",
        "ThiMax_dayless_60",
        "ThiMax_dayless_90",
        "ThiMax_dayless_120",
        "DaysOpen"
    ]
)

h2o.init()

df_h2o = h2o.H2OFrame(df_filter)
df_train, df_validation = df_h2o.split_frame(ratios = [0.8])

targets = [
    "DaysOpen"
]

for target in targets:

    X = [column for column in df_h2o.columns if column not in targets]
    y = target

    aml = H2OAutoML(
        max_runtime_secs = 20 * 3600,
        seed = 42,
        exclude_algos = ["StackedEnsemble"],
        verbosity = "info"
    )

    aml.train(
        x = X,
        y = y,
        training_frame = df_train,
        validation_frame = df_validation
    )

    lb = aml.leaderboard  # get leader board
    print(lb.head(20))
    print("---------")

    df_lb = lb.as_data_frame()
    df_lb.to_csv("lb.csv")
    
    print(f"Best model params: {aml.get_best_model()}")